# Generate Dataset from Your Documentation

![image](./imgs/‎GenAIEnterprises.‎015.png)

![image](./imgs/‎GenAIEnterprises.‎016.png)

# Initial Setup

In [48]:
import boto3
import os
import openai
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from IPython.display import display, Markdown
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

def get_api_key(ssm_client, parameter_path):
    '''Get the OpenAI API key from the SSM Parameter Store'''
    try:
        response = ssm_client.get_parameter(
            Name=parameter_path,
            WithDecryption=True
        )
        return response['Parameter']['Value']
    except ssm_client.exceptions.ParameterNotFound:
        raise Exception(f'Parameter {parameter_path} not found in SSM Parameter Store')

# Create an SSM client using Boto3
region_name = os.getenv('AWS_REGION', 'us-east-1') 
ssm = boto3.client('ssm', region_name=region_name)

openai_api_key = get_api_key(ssm_client=ssm, parameter_path='/openai/api_key')
langchain_api_key = get_api_key(ssm_client=ssm, parameter_path='/langchain/api_key')


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
openai.api_key = openai_api_key

# Set the model variable based on the current date
llm_model = "gpt-3.5-turbo-16k"

# Create the vector store and embedding function
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory='docs/chroma/',
    embedding_function=embedding
)

# Fine-tune the Model

In [50]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "data/qa_pairs_openai.jsonl",
    start_job_id="ftjob-pXSHvTc91Qo0G5yUpVhqu1gY"  # if you have an existing job, can specify id here
)

In [51]:
finetune_engine.get_current_job()

<FineTuningJob fine_tuning.job id=ftjob-pXSHvTc91Qo0G5yUpVhqu1gY at 0x17ada9490> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-pXSHvTc91Qo0G5yUpVhqu1gY",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1694463971,
  "finished_at": 1694465299,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:neurons-lab::7xi7PZeg",
  "organization_id": "org-f5H7bPv9fgPptJoKt2f4cPHG",
  "result_files": [
    "file-Jgqpy0QzqBsVMX2M89b9GcMZ"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-uLoAJjacAessJzY7WEcU2oVX",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 78162,
  "error": null
}

In [52]:
import openai
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x17ada8b90> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-pXSHvTc91Qo0G5yUpVhqu1gY",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1694463971,
      "finished_at": 1694465299,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:neurons-lab::7xi7PZeg",
      "organization_id": "org-f5H7bPv9fgPptJoKt2f4cPHG",
      "result_files": [
        "file-Jgqpy0QzqBsVMX2M89b9GcMZ"
      ],
      "status": "succeeded",
      "validation_file": null,
      "training_file": "file-uLoAJjacAessJzY7WEcU2oVX",
      "hyperparameters": {
        "n_epochs": 3
      },
      "trained_tokens": 78162,
      "error": null
    }
  ],
  "has_more": false
}

# Test the Model Manually
## GPT-3.5 Turbo

In [54]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

messages = [
    SystemMessage(
        content="Provide answers to questions based on the company handbook to help employees quickly find the information they need. Ensure that your responses are concise and directly address the questions asked without providing additional information."
    ),
    HumanMessage(
        content="What is the purpose of a buddy in the Made Tech company?"
    ),
]

llm_model = "gpt-3.5-turbo"

chat = ChatOpenAI(model_name=llm_model, temperature=0)
result = chat(messages)

display(Markdown(result.content))

The purpose of a buddy in Made Tech is to provide support and guidance to new employees during their onboarding process.

## Fine Tuned GPT-3.5 Turbo

In [55]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

messages = [
    SystemMessage(
        content="Provide answers to questions based on the company handbook to help employees quickly find the information they need. Ensure that your responses are concise and directly address the questions asked without providing additional information."
    ),
    HumanMessage(
        content="What is the purpose of a buddy in the Made Tech company?"
    ),
]

llm_model = "ft:gpt-3.5-turbo-0613:neurons-lab::7xi7PZeg"

chat = ChatOpenAI(model_name=llm_model, temperature=0)
result = chat(messages)

display(Markdown(result.content))

The purpose of a buddy in the Made Tech company is to provide support and guidance to new joiners during their first few weeks. Buddies are responsible for helping new joiners settle in, answering any questions they may have, and assisting them in finding the right people to speak to. They also play a role in introducing new joiners to the company culture and values, as well as organizing social activities to help them get to know their team and other members of the company. Overall, buddies serve as a friendly point of contact for new joiners and aim to make their onboarding experience as smooth and enjoyable as possible.

## Fine Tuned GPT-3.5 Turbo with RAG

In [56]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

question = "What is the purpose of a buddy in the Made Tech company?"
llm_model = "ft:gpt-3.5-turbo-0613:neurons-lab::7xi7PZeg"
#llm_model ="gpt-3.5-turbo"

llm = ChatOpenAI(model_name=llm_model, temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={'fetch_k': 5, 'k': 7}),
    return_source_documents=True,
)

result = qa_chain(question)
pretty_print_docs(result["source_documents"])
display(Markdown(result["result"]))

Document 1:

You've been chosen to be a buddy to a new team member - Yay. Here's some guidance to help you find out what's expected.  
A buddy is a friend at Made Tech. Someone friendly who helps new team members connect and find their way, especially in their first 3 months with us when everything is new to them.  
Being a buddy is a really important way to help a new person integrate into Made Tech.
When we're back in the office, you will show them all the good lunch places near the office, take them out for a coffee or lunch and help them find their way in the local area.  
As we're working remotely currently, most or all of your interactions will happen on video calls which we call 'coffee chats'.
----------------------------------------------------------------------------------------------------
Document 2:

At Made Tech, mentorship enables Made Tech staff with the opportunity to provide guidance and support to people in the wider tech community. Mentees are able to gain new knowl

The purpose of a buddy in the Made Tech company is to help new team members connect and find their way, especially in their first 3 months with the company when everything is new to them. Buddies are friendly individuals who provide support and guidance to new team members, both in-person when back in the office (e.g., showing them good lunch places, going out for coffee or lunch, helping them navigate the local area) and remotely through video calls or "coffee chats". Being a buddy is an important role in helping new team members integrate into Made Tech.